#### loading test rigs

In [2]:
from test_rigs import *
%load test_rigs.py
%run test_rigs.py
# mayavi_render_all(env_size,city_map,d_dict,g_dict, o_dict)

[9.5 5.5 9.5]
-- test rigs ready! --


#### Implementing 1-step update
Given the actions for each id, update the corresponding list.

In [8]:
from utils.miscellaneous import *

def impact_check_first(
        num_agents_: int,
        city_map_: np.ndarray,
        x_paths: np.ndarray,
        y_paths: np.ndarray,
        z_paths: np.ndarray,
        time_step: int,
        global_state_: dict):
    """
    Check impact with buildings. This shall be called before checking collision
    with agents.
    Args:
        num_agents_: (int): number of agents.
        city_map_ (np.ndarray): a 2D numpy array for the city.
        x_paths (np.ndarray): interpolation of agents' paths in x-axis.
        y_paths (np.ndarray): interpolation of agents' paths in y-axis.
        z_paths (np.ndarray): interpolation of agents' paths in z-axis.
        time_step (int): current timestep
        global_state_ (dict): global states for possible back-tracking

    Returns:
        a dictionary whose keys are agent ids involved in impact, values are a list with impact point coordinates.
    """
        # check collisions with buildings
    blk_x = [int_dtype(coords_ - coords_ % 1) for coords_ in x_paths]
    blk_y = [int_dtype(coords_ - coords_ % 1) for coords_ in y_paths]
    blk_z = [int_dtype(coords_ - coords_ % 1) for coords_ in z_paths]

    # this dict would have agent id as keys, and value being a list whose
    # values being the check results (1 for collision, 0 for free)
    impact_with_buildings_res = {}
    # assert len(blk_paths) == num_agents

    for id_ in range(num_agents_):
        check_list = []
        for blk_id in range(len(blk_x[id_])):
            _x = blk_x[blk_id]
            _y = blk_y[blk_id]
            _z = blk_z[blk_id]
            if city_map_[_x][_y] >= _z:
                check_list.append(1)
                # no need to check further
                break
            else:
                check_list.append(0)
        impact_with_buildings_res[id_] = check_list

    impact_with_buildings = {}
    for id_ in range(num_agents_):
        if sum(impact_with_buildings_res[id_]) != 0:
            # collision detected
            # if len == 1 need to back-track to previous timestep for pre-impact coords
            if len(impact_with_buildings_res[id_]) < 2:
                impact_idx = len(impact_with_buildings[id_]) - 1
                pre_impact_coords = [
                    global_state_["loc_x"][time_step - 1][id_],
                    global_state_["loc_y"][time_step - 1][id_],
                    global_state_["loc_z"][time_step - 1][id_]
                ]
            else:
                impact_idx = len(impact_with_buildings_res[id_]) - 2
                pre_impact_idx = impact_idx - 1  # useful for resolving
                pre_impact_coords = np.array([
                    x_paths[id_][pre_impact_idx],
                    y_paths[id_][pre_impact_idx],
                    z_paths[id_][pre_impact_idx]
                ], dtype=float_dtype)
            impact_coords = np.array([
                x_paths[id_][impact_idx],
                y_paths[id_][impact_idx],
                z_paths[id_][impact_idx]
            ], dtype=float_dtype)
            impact_with_buildings[id_] = [impact_coords, pre_impact_coords]

    return impact_with_buildings


In [4]:
def collision_check_then(
        _num_agents: int,
        collision_radius: float,
        impact_with_buildings: dict,
        x_paths: np.ndarray,
        y_paths: np.ndarray,
        z_paths: np.ndarray):
    """

    Args:
        _num_agents (int): number of agents
        collision_radius (float): minimum distance for clearance between coords.
        impact_with_buildings (dict): dict contains impact information.
        x_paths (np.ndarray): interpolation of agents' paths in x-axis.
        y_paths (np.ndarray): interpolation of agents' paths in y-axis.
        z_paths (np.ndarray): interpolation of agents' paths in z-axis.

    Returns:
        the updated impact information, and a dictionary whose keys are agent ids involved in collision, values are a list with the other impact point agent ids, and index in paths.

    """
    checked_set = set()
    collision_with_agents_res = {}
    collision_res = {}
    collision_pair_dict = {}

    for ag_id in range(num_agents):
        # init pairs and result lists in dict for new points
        pair = set()
        pair.add(ag_id)
        if ag_id not in collision_with_agents_res.keys():
            collision_with_agents_res[ag_id] = []
        # load the paths
        ag_paths = np.array(
            (x_paths[ag_id], y_paths[ag_id], z_paths[ag_id]),
            dtype=float_dtype
        ).transpose()
        paths_len = ag_paths.shape[0]

        # perform the check for new pairs of points
        for _id in range(num_agents):
            pair.add(_id)
            if ag_id != _id and pair not in checked_set:
                # init the result lists as well
                collision_with_agents_res[_id] = []

                _id_paths = np.array(
                    (x_paths[_id], y_paths[_id], z_paths[_id]),
                    dtype=float_dtype
                ).transpose()
                _id_path_len = _id_paths.shape[0]

                # retrieve impact information for the pair
                if ag_id in impact_with_buildings.keys():
                    ag_id_impact_idx = impact_with_buildings[ag_id][0]
                    ag_id_impact = True
                else:
                    ag_id_impact_idx = np.inf
                    ag_id_impact = False

                if _id in impact_with_buildings.keys():
                    _id_impact_idx = impact_with_buildings[_id][0]
                    _id_impact = True
                else:
                    _id_impact_idx = np.inf
                    _id_impact = False

                check_list_ = []
                for _idx in range(min(paths_len, _id_path_len)):
                    distance = np.linalg.norm(
                        ag_paths[_idx] - _id_paths[_idx]
                    )
                    if distance <= collision_radius * 2:
                        # a collision detected
                        # no need to check for collision given a proceeding impact
                        if _idx <= ag_id_impact_idx and _id <= _id_impact_idx:
                            # valid collision, as impact time is at later or infinity
                            check_list_.append(-1)
                            collision_res[ag_id] = -1
                            collision_res[_id] = -1
                            collision_pair_dict[ag_id] = _id
                            # and nullify the impact later for the pair
                            if ag_id_impact:
                                impact_with_buildings.pop(ag_id)
                            if _id_impact:
                                impact_with_buildings.pop(_id)
                            # no need to check further
                            break
                        else:
                            # one of the pair impacts buildings earlier
                            break
                    else:
                        check_list_.append(0)
                # complete the check for one pair
                checked_set.add(pair)
                pair.remove(_id)
                # update dict for this pair of points
                collision_with_agents_res[ag_id].append(check_list_)
                collision_with_agents_res[_id].append(check_list_)
            else:
                pass

    collision_with_agents = {}
    for ag_id in collision_with_agents_res.keys():
        result = np.array(collision_with_agents_res[ag_id]).sum()
        assert result == -1 or 0
        for res in collision_with_agents_res[ag_id]:
            if sum(res) == -1:
                # need to retrieve the other point involved in collision as well
                # todo: what if len(res) == 1?
                if len(res) < 2:
                    ag_id_idx = len(res) - 1
                else:
                    ag_id_idx = len(res) - 2
                _id = collision_pair_dict[ag_id]
                _id_res_arr = np.array(collision_with_agents_res[_id])
                assert _id_res_arr.sum() == -1
                _id_idx = ag_id_idx
                ag_id_coords = np.array([
                    x_paths[ag_id][ag_id_idx],
                    y_paths[ag_id][ag_id_idx],
                    z_paths[ag_id][ag_id_idx]
                ], dtype=float_dtype)
                collision_with_agents[ag_id] = [_id, ag_id_coords]
                if _id not in collision_with_agents.keys():
                    # new
                    collision_with_agents_res.pop(_id)
                    _id_coords = np.array([
                    x_paths[_id][_id_idx],
                    y_paths[_id][_id_idx],
                    z_paths[_id][_id_idx]
                ], dtype=float_dtype)
                    collision_with_agents[_id] = [ag_id, _id_coords]

    return impact_with_buildings, checked_set, collision_with_agents


In [5]:
def resolve_impact(
        impact_damping: float,
        impact_with_buildings: dict,
        loc_x_curr_t: np.ndarray,
        loc_y_curr_t: np.ndarray,
        loc_z_curr_t: np.ndarray,
        speed_curr_t: np.ndarray,
        vert_speed_curr_t: np.ndarray,
        dir_curr_t: np.ndarray):
    """
    Flip agents current direction and reduce speeds by the damping ratio upon
    impact with buildings.
    Args:
        impact_damping:
        impact_with_buildings:
        loc_x_curr_t:
        loc_y_curr_t:
        loc_z_curr_t:
        speed_curr_t:
        vert_speed_curr_t:
        dir_curr_t:

    Returns:
        Updated speed, direction and locations for all agents.

    """
    impact_agent_ids = [ ag_id for ag_id in impact_with_buildings.keys()]
    impact_coords = [
        impact_with_buildings[ag_id][0] for ag_id in impact_with_buildings.keys()
    ]

    pre_impact_coords = [
        impact_with_buildings[ag_id][1] for ag_id in impact_with_buildings.keys()
    ]
    # get the borders crossed
    crossed_blk_borders = np.rint(pre_impact_coords, dtype=int_dtype)

     # mirror the coordinates along the crossed borders
    after_impact_coords = 2 * impact_coords - crossed_blk_borders

    # update the location
    loc_x_curr_t[impact_agent_ids] = after_impact_coords[0]
    loc_y_curr_t[impact_agent_ids] = after_impact_coords[1]
    loc_z_curr_t[impact_agent_ids] = after_impact_coords[2]

    loc_curr_t = (loc_x_curr_t, loc_y_curr_t, loc_z_curr_t)

    # damp the speed
    speed_curr_t[impact_agent_ids] =  impact_damping* speed_curr_t[impact_agent_ids]
    vert_speed_curr_t[impact_agent_ids] = \
        impact_damping * vert_speed_curr_t[impact_agent_ids]

    speed_updated = (speed_curr_t, vert_speed_curr_t)
    # flip direction...
    x_dir = np.cos(dir_curr_t)
    y_dir = np.sin(dir_curr_t)

    x_dir[impact_agent_ids] = - x_dir[impact_agent_ids]
    y_dir[impact_agent_ids] = - y_dir[impact_agent_ids]

    # get back the direction in radian
    dir_curr_t = np.arctan2(x_dir, y_dir)

    return speed_updated, dir_curr_t, loc_curr_t


In [6]:
def resolve_collision(
        collision_pair_set: set,
        collision_with_agents: dict,
        speed_curr_t: np.ndarray,
        vert_speed_curr_t: np.ndarray,
        dir_curr_t: np.ndarray):

    agent_ids = [ ag_id for ag_id in collision_with_agents.keys()]

    collision_pair_list = [tuple(pair) for pair in collision_pair_set]

    v_x = speed_curr_t[agent_ids] * np.cos(dir_curr_t[agent_ids])
    v_y = speed_curr_t[agent_ids] * np.sin(dir_curr_t[agent_ids])
    v_z = vert_speed_curr_t[agent_ids]

    # we would resolve the collision for each pair, instead of each agent.

    # for each pair of colliders, get their delta distances and velocities
    for pair in collision_pair_list:
        # determine the 3D angle between colliders
        _1_id = pair[0]
        _2_id = pair[1]

        _1_delta_r = np.array(
            collision_with_agents[_1_id][1] - collision_with_agents[_2_id][1],
            dtype=float_dtype)
        _2_delta_r = np.array(
            collision_with_agents[_2_id][1] - collision_with_agents[_1_id][1],
            dtype=float_dtype)

        _1_velocity = np.array(
            (v_x[_1_id], v_y[_1_id], v_z[_1_id]), dtype=float_dtype)
        _2_velocity = np.arrat(
            (v_x[_1_id], v_y[_1_id], v_z[_1_id]), dtype=float_dtype
        )

        _1_angle = np.arccos(
            np.dot(_1_velocity, _1_delta_r) /
            (np.linalg.norm(_1_velocity) * np.linalg.norm(_1_delta_r)),
            dtype=float_dtype)
        _2_angle = np.arccos(
            np.dot(_2_velocity, _2_delta_r) /
            (np.linalg.norm(_2_velocity) * np.linalg.norm(_2_delta_r)),
            dtype=float_dtype)

        # calculate colliders' velocity vectors towards each other
        _1_center_vel = _1_velocity * np.cos(_1_angle)
        _2_center_vel = _2_velocity * np.cos(_2_angle)

        # decompose vector into x-y-z components(?)

        # determine velocity normal to the center-line
        _1_normal_vel = _1_velocity - _1_center_vel
        _2_normal_vel = _2_velocity - _2_center_vel

        # switch the colliders' velocity vectors
        temp = _1_center_vel
        _1_center_vel = _2_center_vel
        _2_center_vel = temp

        # compose new vectors into new velocity
        _1_new_velocity = _1_normal_vel + _1_center_vel
        _2_new_velocity = _2_normal_vel + _2_center_vel

        # update direction, speed, vertical speed
        _1_new_dir = np.arctan2(_1_new_velocity[1], _1_new_velocity[0], dtype=float_dtype)
        _2_new_dir = np.arctan2(_2_new_velocity[1], _2_new_velocity[0], dtype=float_dtype)
        _1_new_speed = np.sqrt(
            _1_new_velocity[0]**2 + _1_new_velocity[1]**2, dtype=float_dtype
        )
        _2_new_speed = np.sqrt(
            _2_new_velocity[0]**2 + _2_new_velocity[1]**2, dtype=float_dtype
        )
        _1_new_v_speed = float_dtype(_1_new_velocity[2])
        _2_new_v_speed = float_dtype(_2_new_velocity[2])

        speed_curr_t[_1_id] = _1_new_speed
        speed_curr_t[_2_id] = _2_new_speed
        vert_speed_curr_t[_1_id] = _1_new_v_speed
        vert_speed_curr_t[_2_id] = _2_new_v_speed
        dir_curr_t[_1_id] = _1_new_dir
        dir_curr_t[_2_id] = _2_new_dir

        # it seems that we do not need to touch the location...
    return speed_curr_t, vert_speed_curr_t, dir_curr_t


In [7]:
def impact_and_collision(
        time_step: int,
        _num_agents_: int,
        city_map_: np.ndarray,
        global_state_: dict,
        loc_x_curr_t_: np.ndarray,
        loc_y_curr_t_: np.ndarray,
        loc_z_curr_t_: np.ndarray,
        speed_curr_t_: np.ndarray,
        vert_speed_curr_t_: np.ndarray,
        dir_curr_t_: np.ndarray,
        collision_radius: float = .2 * spacing_for_drones):
    """
    Given the current time-step and all agent's state, check for impact with buildings and collision within agents.
    Args:
        time_step: current time-step.
        _num_agents_: total number of agents
        city_map_:
        global_state_:
        loc_x_curr_t_:
        loc_y_curr_t_:
        loc_z_curr_t_:
        speed_curr_t_:
        vert_speed_curr_t_:
        dir_curr_t_:
        collision_radius:

    Returns:
        Updated locations, kinematics, and agents ids involved in impact and collision.

    """
    # todo: might need to tune the collision radius
    # collision_radius = .2 * spacing_for_drones
    # note that all the list/array indices are representing the agent id
    loc_x_prev_t_ = global_state_["loc_x"][time_step - 1]
    loc_y_prev_t_ = global_state_["loc_y"][time_step - 1]
    loc_z_prev_t_ = global_state_["loc_z"][time_step - 1]

    # linear interpolation of agents' paths with constant spacing of 1/2 collision radius
    x_paths = np.array(
        [np.arange(loc_x_prev_t_[id_], loc_x_curr_t_[id_], collision_radius / 2, dtype=float_dtype)] for id_ in range(num_agents))

    y_paths = np.array(
        [np.arange(loc_y_prev_t_[id_], loc_y_curr_t_[id_], collision_radius / 2,dtype=float_dtype)] for id_ in range(num_agents))

    z_paths = np.array(
        [np.arange(loc_z_prev_t_[id_], loc_z_curr_t_[id_], collision_radius / 2, dtype=float_dtype)] for id_ in range(num_agents))

    # check collisions with buildings
    impact_with_buildings = \
        impact_check_first(
            _num_agents_,
            city_map_,
            x_paths, y_paths, z_paths,
            global_state_=global_state_,
            time_step=time_step)

    impact_with_buildings, collision_set, collision_with_agents = \
        collision_check_then(
            _num_agents_,
            collision_radius,
            impact_with_buildings,
            x_paths, y_paths, z_paths)

    # resolving impact
    (speed_curr_t_, vert_speed_curr_t_), dir_curr_t, (loc_x_curr_t_, loc_y_curr_t_, loc_z_curr_t_) = \
        resolve_impact(
            impact_damping=0.5,
            impact_with_buildings=impact_with_buildings,
            loc_x_curr_t=loc_x_curr_t_,
            loc_y_curr_t=loc_y_curr_t_,
            loc_z_curr_t=loc_z_curr_t_,
            speed_curr_t=speed_curr_t_,
            vert_speed_curr_t=vert_speed_curr_t_,
            dir_curr_t=dir_curr_t_)

    # resolving collision
    speed_curr_t_, vert_speed_curr_t_, dir_curr_t_ = \
        resolve_collision(
            collision_pair_set=collision_set,
            collision_with_agents=collision_with_agents,
            speed_curr_t=speed_curr_t_,
            vert_speed_curr_t=vert_speed_curr_t_,
            dir_curr_t=dir_curr_t)

    # todo: return these agents id for calculating rewards
    impact_agent_ids = [ ag_id for ag_id in impact_with_buildings.keys()]
    collision_agent_ids = [ ag_id for ag_id in collision_with_agents.keys()]


    # todo: in production code, we will just update global state dictionary
    return (loc_x_curr_t_, loc_y_curr_t_, loc_z_curr_t_), (speed_curr_t_, vert_speed_curr_t_, dir_curr_t_), impact_agent_ids, collision_agent_ids


In [ ]:
def update_state_test(global_state, time_step, delta_accelerations, delta_turns, delta_vert):

    loc_x_prev_t = global_state["loc_x"][time_step - 1]
    loc_y_prev_t = global_state["loc_y"][time_step - 1]
    loc_z_prev_t = global_state["loc_z"][time_step - 1]
    speed_prev_t = global_state["speed"][time_step - 1]
    dir_prev_t = global_state["direction"][time_step - 1]
    acc_prev_t = global_state["acceleration"][time_step - 1]
    vert_speed_prev_t = global_state["vertical speed"][time_step - 1]
    vert_acc_prev_t = global_state["vertical acceleration"][time_step - 1]

    # update direction and acceleration
    # no update if agent is out of game (only for multi-goals)
    dir_curr_t = (
        (dir_prev_t + delta_turns) % (2 * np.pi) * still_in_the_game
    ).astype(float_dtype)

    acc_curr_t = acc_prev_t + delta_accelerations

    max_speed = np.array(
        [
            max_drone_speed if agent_type[_id] == 1
            else max_goal_speed if agent_type[_id] == 0
            else max_obstacle_speed
            for _id in range(num_agents)
        ], dtype=float_dtype
    )

    speed_curr_t = float_dtype(
        np.clip(speed_prev_t + acc_curr_t, 0.0, max_speed) * still_in_the_game
    )

    # reset acceleration to 0 when over-speed
    acc_curr_t = float_dtype(
        acc_curr_t * (speed_curr_t > 0) * (speed_curr_t < max_speed)
    )

    # vertical ones
    vert_acc_curr_t = vert_acc_prev_t + delta_vert

    max_vert_speed = np.array(
        [
            max_drone_vert_speed if agent_type[_id] == 1
            else max_goal_speed if agent_type[_id] == 0
            else max_obstacle_speed
            for _id in range(num_agents)
        ], dtype=float_dtype
    )

    vert_speed_curr_t = float_dtype(
        np.clip(vert_speed_prev_t + vert_acc_curr_t, 0.0, max_vert_speed)
    )

    # reset acceleration to 0 when over-speed
    vert_acc_curr_t = float_dtype(
        acc_curr_t * (vert_speed_curr_t > 0) * (speed_curr_t < max_vert_speed)
    )

    # update the temporary locations
    loc_x_curr_t = float_dtype(
        loc_x_prev_t + speed_curr_t * np.cos(dir_curr_t)
    )

    loc_y_curr_t = float_dtype(
        loc_y_prev_t + speed_curr_t * np.sin(dir_curr_t)
    )

    loc_z_curr_t = float_dtype(
        loc_z_prev_t + vert_speed_curr_t
    )

    # Crossing the edge
    # todo: can we get the agent ids that have crossed the edge?
    has_cross_edge = ~(
        (loc_x_curr_t >= 0)
        & (loc_x_curr_t <= env_size)
        & (loc_y_curr_t >= 0)
        & (loc_y_curr_t <= env_size)
        & (loc_z_curr_t >= 0)
        & (loc_z_curr_t <= env_size)
    )

    # Clip x, y and z if agent has crossed edge
    clipped_loc_x_curr_t = float_dtype(np.clip(loc_x_curr_t, 0.0, env_size))

    clipped_loc_y_curr_t = float_dtype(np.clip(loc_y_curr_t, 0.0, env_size))

    clipped_loc_z_curr_t = float_dtype(np.clip(loc_z_curr_t, 0.0, env_max_height))

    # todo: check collisions and impacts, resolve them, and update info
    loc_curr_t, kinematics_curr_t, impact_agent_ids, collision_agent_ids = \
        impact_and_collision(
            time_step=time_step,
            _num_agents_=num_agents,
            city_map_=city_map,
            global_state_=global_state,
            loc_x_curr_t_=clipped_loc_x_curr_t,
            loc_y_curr_t_=clipped_loc_y_curr_t,
            loc_z_curr_t_=clipped_loc_z_curr_t,
            speed_curr_t_=speed_curr_t,
            vert_speed_curr_t_=vert_speed_curr_t,
            dir_curr_t_=dir_curr_t)

    # todo: unpack and update locations and kinematics
    new_loc_x_curr_t, new_loc_y_curr_t, new_loc_z_curr_t = loc_curr_t
    new_speed_curr_t, new_vert_speed_curr_t, new_dir_curr_t = kinematics_curr_t

    # set global states
    set_global_state(global_state, key="loc_x", value=new_loc_x_curr_t, t=time_step)
    set_global_state(global_state, key="loc_y", value=new_loc_y_curr_t, t=time_step)
    set_global_state(global_state, key="loc_z", value=new_loc_z_curr_t, t=time_step)
    set_global_state(global_state, key="speed", value=new_speed_curr_t, t=time_step)
    set_global_state(global_state, key="vertical speed", value=new_vert_speed_curr_t, t=time_step)
    set_global_state(global_state, key="direction", value=new_dir_curr_t, t=time_step)
    set_global_state(global_state, key="acceleration", value=acc_curr_t, t=time_step)
    set_global_state(global_state, key="vertical acceleration", value=vert_acc_curr_t, t=time_step)

    return impact_agent_ids, collision_agent_ids, has_cross_edge


In [3]:
def step_test(time_step, num_agents, num_drones, agent_type, global_state, actions_):
    time_step += 1
    # print(actions_[0])
    acceleration_action_ids = [actions_[agent_id][0] for agent_id in range(num_agents)]

    turn_action_ids = [actions_[agent_id][2] for agent_id in range(num_agents)]

    vert_acceleration_action_ids = [actions_[agent_id][1] for agent_id in range(num_agents)]

    assert all(
        0 <= acc <= num_acceleration_levels
        for acc in acceleration_action_ids
    )

    assert all(
        0 <= turn <= num_turn_levels
        for turn in turn_action_ids
    )

    assert all(
        0 <= vert_acc <= num_acceleration_levels
        for vert_acc in vert_acceleration_action_ids
    )

    delta_accelerations = acceleration_actions[acceleration_action_ids]
    delta_turns = turn_actions[turn_action_ids]
    delta_vert = vert_acceleration_actions[vert_acceleration_action_ids]

    # todo: then update the state
    impact_info, collision_info, cross_edge_info = \
        update_state_test(global_state,time_step,delta_accelerations,delta_turns,delta_vert)

    # todo: compute rewards
    rew_list = compute_rewards(
        t = time_step,
        _flock_rew_coef = 0.5,
        _flock_threshold = 0.5,
        _track_rew_coef = 0.5,
        _stab_pen_coef = 0.5,
        _impact_penalties = 0.5,
        _collision_penalties = 0.5,
        _cross_edge_penalties = 0.5,
        _step_penalties = 1,
        _num_agents = num_agents,
        _num_drones = num_drones,
        _agent_type = agent_type,
        _impact_info = impact_info,
        _collision_info = collision_info,
        _cross_edge_info = cross_edge_info,
        _global_state = global_state
    )

    # todo: the reward is part of the observation

    # todo: generate observation


/home/hibiki/miniconda3/envs/opencv4/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Naive stepping and rendering test (no collision, rewards, or observation)

In [5]:
from utils.rendering import *

In [6]:
test_steps = 10
step = 0

mock_city_map = np.zeros((env_size, env_size))

while step < test_steps:
    # sample actions for all agents
    action_dict = {agent_id: action_space[agents_id].sample() for agent_id in range(num_agents)}
    # print(action_dict)
    # step
    step_test(step, action_dict)
    # mayavi_render_all(env_size, )
    step += 1

In [ ]:
mayavi_render_ffmpeg_video(
    size=env_size,
    max_time_step=test_steps,
    num_agents=num_agents,
    city=city_map,
    agents_type=agent_type,
    agents_state=global_state,
    cam_azimuth=-60,
    cam_elevation=45,
    cam_distance=70,
    cam_foc=(10, 10, 10)
)

#### calculating drones and goals distances

In [9]:
def compute_rewards(
        t: int,
        _flock_rew_coef: float,
        _flock_threshold: float,
        _track_rew_coef: float,
        _stab_pen_coef: float,
        _impact_penalties: float,
        _collision_penalties: float,
        _cross_edge_penalties: float,
        _step_penalties: float,
        _num_agents: int,
        _num_drones: int,
        _agent_type: dict,
        _impact_info: dict,
        _collision_info: dict,
        _cross_edge_info: np.ndarray,
        _global_state: dict):
    """
    Compute the rewards based on given information.
    Args:
        t: current time-step. For accessing the global states and back-tracking.
        _flock_rew_coef: coefficient for flocking reward.
        _flock_threshold: threshold for flocking.
        _track_rew_coef: coefficient for drone tracking reward.
        _stab_pen_coef: coefficient for drone stability penalties.
        _impact_penalties: penalties given when agents impact with buildings.
        _collision_penalties: penalties given when agents collide with each others.
        _cross_edge_penalties: penalties given when agents cross the borders.
        _step_penalties: step penalties or rewards for drones or goals.
        _num_agents: number of agents.
        _num_drones: number of drones.
        _agent_type: agent type dictionary
        _impact_info: impact dictionary whose keys are agent ids with impact.
        _collision_info: collision dictionary whose keys are agent ids with collision.
        _cross_edge_info: a numpy array that contains boolean values of whether agents of given index id is still in game.
        _global_state: global state dictionary.

    Returns:
        a reward dictionary whose keys are agent ids, values are rewards received in this step.

    """
    # create unified reward dictionary, in case we need to input rewards for goals.
    # some of the rewards need previous states, so need to handle the case where t == 0.
    p_t = np.max(t - 1, 0)

    _rew = {agent_id: 0.0 for agent_id in range(_num_agents)}

    _drones_set = sorted([ag_id for ag_id in range(num_agents) if _agent_type[ag_id] == 1])

    drones_x = _global_state['loc_x'][t][_drones_set]
    drones_y = _global_state['loc_y'][t][_drones_set]
    drones_z = _global_state['loc_z'][t][_drones_set]

    drones_x_prev = _global_state['loc_x'][p_t][_drones_set]
    drones_y_prev = _global_state['loc_y'][p_t][_drones_set]
    drones_z_prev = _global_state['loc_z'][p_t][_drones_set]

    # todo: compute each drone's distance to flk centroid, and assign reward
    centroid_distances_curr = np.sqrt(
                (np.repeat(drones_x.sum() / _num_drones, num_drones) - drones_x)**2 +
                (np.repeat(drones_y.sum() / _num_drones, num_drones) - drones_y)**2 +
                (np.repeat(drones_z.sum() / _num_drones, num_drones) - drones_z)**2
    ).reshape(1, _num_drones)

    centroid_distances_prev = np.sqrt(
                (np.repeat(drones_x_prev.sum() / _num_drones, num_drones) - drones_x_prev)**2 +
                (np.repeat(drones_x_prev.sum() / _num_drones, num_drones) - drones_y_prev)**2 +
                (np.repeat(drones_x_prev.sum() / _num_drones, num_drones) - drones_z_prev)**2
    ).reshape(1, _num_drones)

    r_flock = np.maximum(centroid_distances_prev - flock_threshold,flock_threshold) - \
              np.maximum(centroid_distances_curr - flock_threshold, flock_threshold)

    for idx in range(num_drones):
        _rew[_drones_set[idx]] += _flock_rew_coef * r_flock[idx]

    # todo: compute each drone's distance to goals, use the delta value to assign reward
    # in the case of multiple goals, need to verify if the goal is still active or not
    _goals_list = sorted([ag_id for ag_id in range(_num_agents) if _agent_type[ag_id] == 0
                   and _global_state["still in the game"][t][ag_id] == 1])

    _num_goals = len(_goals_list)

    goals_x = _global_state["loc_x"][t][_goals_list]
    goals_y = _global_state["loc_y"][t][_goals_list]
    goals_z = _global_state["loc_z"][t][_goals_list]

    goals_x_prev = _global_state["loc_x"][p_t][_goals_list]
    goals_y_prev = _global_state["loc_y"][p_t][_goals_list]
    goals_z_prev = _global_state["loc_z"][p_t][_goals_list]

    goals_distances_curr = np.sqrt(
        (np.repeat(goals_x, _num_drones) - np.tile(drones_x, _num_goals)) ** 2 +
        (np.repeat(goals_y, _num_drones) - np.tile(drones_y, _num_goals)) ** 2 +
        (np.repeat(goals_z, _num_drones) - np.tile(drones_z, _num_goals)) ** 2
    ).reshape(_num_drones, _num_goals)

    goals_distances_prev = np.sqrt(
        (np.repeat(goals_x_prev, _num_drones) - np.tile(drones_x_prev, _num_goals)) ** 2 +
        (np.repeat(goals_y_prev, _num_drones) - np.tile(drones_y_prev, _num_goals)) ** 2 +
        (np.repeat(goals_z_prev, _num_drones) - np.tile(drones_z_prev, _num_goals)) ** 2
    ).reshape(_num_drones, _num_goals)

    # in the case of multiple goals, drones would only receive rewards from the closest goal
    # get the tracked goal id for each drone from the previous distances
    # then fetch the curr distances to this goal for each drone as well
    goals_tracked = np.argmin(goals_distances_prev, axis=1)
    goals_track_prev = np.diag(goals_distances_prev[:, goals_tracked])
    goals_track_curr = np.diag(goals_distances_curr[:, goals_tracked])

    r_track = goals_track_curr - goals_track_prev

    assert r_track.shape[0] == _num_drones

    for idx in range(num_drones):
        _rew[_drones_set[idx]] += _track_rew_coef * r_track[idx]

    # todo: penalties for impact and collision, do it for all agents
    for idx in range(_num_agents):
        if idx in _impact_info.keys():
            _rew[_agent_type[idx]] -= _impact_penalties

        if idx in _collision_info.keys():
            _rew[_agent_type[idx]] -= _collision_penalties

    # todo: penalties for crossing the edge, do it for all agents
    _cross_edge_penalties_list =  _cross_edge_penalties * _cross_edge_info
    assert _cross_edge_penalties_list.shape[0] == _num_agents
    for idx in range(_num_agents):
        _rew[_agent_type[idx]] = _cross_edge_penalties_list[idx]

    # todo: penalties for unstable behaviours, calculated using delta values
    speed_curr_t = _global_state["speed"][t][_drones_set]
    v_speed_curr_t = _global_state["vertical speed"][t][_drones_set]
    velocity_curr_t = np.sqrt(speed_curr_t ** 2 + v_speed_curr_t ** 2)

    speed_prev_t = _global_state["speed"][p_t][_drones_set]
    v_speed_prev_t = _global_state["vertical speed"][p_t][_drones_set]
    velocity_prev_t = np.sqrt(speed_prev_t **2 + v_speed_prev_t ** 2)

    p_stability = np.abs(velocity_prev_t - velocity_curr_t)

    for idx in range(num_drones):
        _rew[_drones_set[idx]] -= _stab_pen_coef * p_stability

    # todo: stepping penalties / rewards for drones or goals that are still not captured.
    for idx in range(_num_agents):
        if _agent_type[idx] == 1:
            _rew[_agent_type[idx]] -= _step_penalties
        elif _agent_type[idx] == 0 and _global_state["still in the game"][t][_agent_type[idx]]:
            _rew[_agent_type[idx]] += _step_penalties

    return _rew


In [45]:
# drones_set = [ag_id for ag_id in range(num_agents) if agent_type[ag_id] == 1]
# drones_x = global_state["loc_x"][0][drones_set]
# drones_y = global_state["loc_y"][0][drones_set]
# drones_z = global_state["loc_z"][0][drones_set]
#
# drones_flock_x = drones_x.sum() / num_drones
# drones_flock_y = drones_y.sum() / num_drones
# drones_flock_z = drones_z.sum() / num_drones

In [11]:
# the following are logic from compute reward function
# now we only compute the rewards for drones
# but we create a dictionary for everyone anyway
rew = {agent_id: 0.0 for agent_id in range(num_agents)}
# here the sorted method directly returns a list from the dictionary
drones_list = sorted(d_dict)

goals_list = sorted(g_dict)
goals_locations_x = global_state["loc_x"][timestep][goals_list]
drones_locations_x = global_state["loc_x"][timestep][drones_list]

goals_locations_y = global_state["loc_y"][timestep][goals_list]
drones_locations_y = global_state["loc_y"][timestep][drones_list]

goals_locations_z = global_state["loc_z"][timestep][goals_list]
drones_locations_z = global_state["loc_z"][timestep][drones_list]



In [47]:
def compute_distance(agent1, agent2):
    """
    Note: 'compute_distance' is only used when running on CPU step() only.
    When using the CUDA step function, this Python method (compute_distance)
    is also part of the step() function!
    """
    return np.sqrt(
        (
                global_state["loc_x"][timestep, agent1]
                - global_state["loc_x"][timestep, agent2]
        )
        ** 2
        + (
                global_state["loc_y"][timestep, agent1]
                - global_state["loc_y"][timestep, agent2]
        )
        ** 2
        + (
                global_state["loc_z"][timestep, agent1]
                - global_state["loc_z"][timestep, agent2]
        )
        ** 2
    ).astype(float_dtype)

In [12]:
# this would return an 2D array
# each row for a goal
# with each entry inside the row for distance between this goal and the corresponding drones
goals_to_drones_distances = np.sqrt(
                (
                    np.repeat(goals_locations_x, num_drones)
                    - np.tile(drones_locations_x, num_goals)
                )
                ** 2
                + (
                    np.repeat(goals_locations_y, num_drones)
                    - np.tile(drones_locations_y, num_goals)
                )
                ** 2
                + (
                    np.repeat(goals_locations_z, num_drones)
                    - np.tile(drones_locations_z, num_goals)
                )
                ** 2
            ).reshape(num_goals, num_drones)

# todo: actually we need the next location to calculate the rewards
# we can directly use the indices of each row entry to retrieve drones id from drones_list
# as drones_list and drones_locations are sorted
# computing the pursuing rewards
# for drones_idx in range(num_drones):
#     agent_id = drones_list[drones_idx]
#     rew[agent_id] += pursuit_rewards_coef * goals_to_drones_distances_prev
# for agent_id in range(num_agents):
#     if still_in_the_game[agent_id]:
#         rew[agent_id] += pursuit_rewards_coef * goals_to_drones_distances

In [24]:
import numpy as np
a = [[1, 2],
 [5, 3],
 [3, 1],
 [4, 3],
 [5, 4],
 [6, 7]]

g_ = np.array(a)
track = np.argmin(g_, axis=1)
print(track)
tracked = np.diag(g_[:, track])
print(tracked)

[0 1 1 1 1 0]
[1 3 1 3 4 6]


In [43]:
# has_crossed_edge = ~(
#             (loc_x_curr_t >= 0)
#             & (loc_x_curr_t <= self.grid_length)
#             & (loc_y_curr_t >= 0)
#             & (loc_y_curr_t <= self.grid_length)
#         )

l_x = np.random.random(10)
l_y = np.random.random(10)
l_z = np.random.random(10)

check = ~(
    (l_x >= 0.01)
    & (l_x <= 0.99)
    & (l_y >= 0.01)
    & (l_y <= 0.99)
    & (l_z >= 0.01)
    & (l_z <= 0.99)
)
print(check)

[False False False False False False False False False False]


In [49]:
isinstance(check, tuple)
print(type(check))

<class 'numpy.ndarray'>
